# 📈 技术指标开发: {indicator_name}
**开发日期**: {date}  
**开发者**: {developer}  
**指标类型**: {category} (趋势/动量/波动率/成交量)  
**复杂度**: {complexity} (简单/中等/复杂)

## 🎯 指标概述
- **用途**: 描述指标的主要用途和应用场景
- **计算原理**: 指标的数学原理和计算方法
- **参数说明**: 关键参数及其影响
- **信号解读**: 如何解读指标信号

## 📊 技术规格
- **输入数据**: OHLCV中需要哪些数据
- **输出格式**: Series/DataFrame及列名
- **默认参数**: 推荐的默认参数值
- **计算复杂度**: O(n) / O(n²) 等

In [ ]:
# 技术指标开发环境
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

# 导入现有指标库
from core.strategy.technical_indicators import TechnicalIndicators

# 设置图表样式
plt.style.use('seaborn-v0_8')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_palette("husl")

print(f"🔬 指标开发环境已初始化: {datetime.now()}")
print(f"📁 项目根目录: {project_root}")

## 🧪 测试数据准备

In [ ]:
def create_synthetic_data(days=252, seed=42):
    """创建合成测试数据"""
    np.random.seed(seed)
    dates = pd.date_range('2024-01-01', periods=days, freq='D')
    
    # 生成价格序列 (几何布朗运动)
    returns = np.random.normal(0.0008, 0.02, days)  # 日收益率
    prices = [100]  # 初始价格
    
    for ret in returns[1:]:
        prices.append(prices[-1] * (1 + ret))
    
    # 生成OHLC数据
    data = pd.DataFrame(index=dates)
    data['close'] = prices
    
    # High = Close + 随机上影线
    data['high'] = data['close'] * (1 + np.abs(np.random.normal(0, 0.015, days)))
    
    # Low = Close - 随机下影线  
    data['low'] = data['close'] * (1 - np.abs(np.random.normal(0, 0.015, days)))
    
    # Open接近前日收盘价
    data['open'] = data['close'].shift(1) * (1 + np.random.normal(0, 0.005, days))
    data['open'].iloc[0] = data['close'].iloc[0]
    
    # 确保OHLC逻辑正确
    data['high'] = np.maximum.reduce([data['open'], data['high'], data['low'], data['close']])
    data['low'] = np.minimum.reduce([data['open'], data['high'], data['low'], data['close']])
    
    # 成交量 (与波动率相关)
    volatility = data['close'].pct_change().rolling(20).std()
    base_volume = 1000000
    data['volume'] = (base_volume * (1 + volatility * 10) * 
                     np.random.uniform(0.5, 1.5, days)).astype(int)
    
    return data[['open', 'high', 'low', 'close', 'volume']]

def load_real_data(symbol='000300', days=252):
    """加载真实市场数据 (如果可用)"""
    try:
        # TODO: 从数据源加载真实数据
        # from core.data.data_manager import DataManager
        # dm = DataManager()
        # return dm.get_stock_data([symbol], days=days)
        
        print("⚠️ 真实数据加载功能待实现，使用合成数据")
        return create_synthetic_data(days)
    except Exception as e:
        print(f"⚠️ 无法加载真实数据: {e}，使用合成数据")
        return create_synthetic_data(days)

# 创建测试数据
test_data = create_synthetic_data(252)
print(f"✅ 测试数据已创建: {test_data.shape}")
print(f"📅 数据期间: {test_data.index.min()} 至 {test_data.index.max()}")

# 显示数据概览
test_data.head()

## 💻 指标实现

In [ ]:
def new_indicator(data: pd.Series, param1: int = 14, param2: float = 2.0) -> pd.Series:
    """
    新技术指标实现
    
    Args:
        data: 输入数据序列 (通常是收盘价)
        param1: 参数1说明
        param2: 参数2说明
        
    Returns:
        pd.Series: 指标值序列
        
    Example:
        >>> result = new_indicator(data['close'], param1=10, param2=1.5)
    """
    
    # TODO: 在这里实现你的指标逻辑
    # 示例: 简单移动平均
    result = data.rolling(window=param1).mean()
    
    return result

# 测试指标实现
try:
    indicator_result = new_indicator(test_data['close'])
    print(f"✅ 指标计算成功")
    print(f"📊 结果形状: {indicator_result.shape}")
    print(f"🔢 数据类型: {type(indicator_result)}")
    print(f"📈 有效值数量: {indicator_result.count()}/{len(indicator_result)}")
except Exception as e:
    print(f"❌ 指标计算失败: {e}")

## ✅ 验证测试

In [ ]:
class IndicatorValidator:
    """技术指标验证器"""
    
    def __init__(self, indicator_func):
        self.indicator_func = indicator_func
        self.test_results = {'passed': 0, 'failed': 0, 'errors': []}
    
    def test_basic_functionality(self, data):
        """基础功能测试"""
        try:
            result = self.indicator_func(data['close'])
            
            # 检查返回类型
            assert isinstance(result, (pd.Series, pd.DataFrame)), "返回类型应为Series或DataFrame"
            
            # 检查长度
            assert len(result) == len(data), "指标长度应与输入数据长度一致"
            
            # 检查数值有效性
            if isinstance(result, pd.Series):
                assert not result.isna().all(), "不应该全部为NaN"
                assert not np.isinf(result.dropna()).any(), "不应该包含无穷值"
            
            self.test_results['passed'] += 1
            print("✅ 基础功能测试通过")
            return True
            
        except Exception as e:
            self.test_results['failed'] += 1
            self.test_results['errors'].append(f"基础功能: {e}")
            print(f"❌ 基础功能测试失败: {e}")
            return False
    
    def test_edge_cases(self, data):
        """边界条件测试"""
        edge_cases = [
            ('空数据', pd.Series(dtype=float)),
            ('单个值', pd.Series([100.0])),
            ('重复值', pd.Series([100.0] * 10)),
            ('包含NaN', pd.Series([100, np.nan, 102, 101, np.nan])),
            ('极小数据', data['close'].head(5))
        ]
        
        passed = 0
        for case_name, test_data in edge_cases:
            try:
                result = self.indicator_func(test_data)
                print(f"✅ {case_name}: 通过")
                passed += 1
            except Exception as e:
                print(f"⚠️ {case_name}: {e}")
                # 边界条件失败不算严重错误
        
        self.test_results['passed'] += passed
        print(f"🔍 边界测试: {passed}/{len(edge_cases)} 通过")
        return passed
    
    def test_performance(self, data, iterations=10):
        """性能测试"""
        times = []
        for _ in range(iterations):
            start = time.time()
            try:
                self.indicator_func(data['close'])
                times.append(time.time() - start)
            except:
                continue
        
        if times:
            avg_time = np.mean(times)
            std_time = np.std(times)
            
            print(f"⚡ 性能测试结果:")
            print(f"   平均时间: {avg_time:.4f}s ± {std_time:.4f}s")
            print(f"   测试次数: {len(times)}")
            
            # 性能评级
            if avg_time < 0.001:
                grade = "🟢 优秀"
            elif avg_time < 0.01:
                grade = "🟡 良好" 
            elif avg_time < 0.1:
                grade = "🟠 一般"
            else:
                grade = "🔴 需优化"
                
            print(f"   性能等级: {grade}")
            
            self.test_results['passed'] += 1
            return avg_time
        else:
            print("❌ 性能测试失败")
            self.test_results['failed'] += 1
            return None
    
    def generate_report(self):
        """生成测试报告"""
        total = self.test_results['passed'] + self.test_results['failed']
        success_rate = self.test_results['passed'] / total if total > 0 else 0
        
        print("\n📋 验证测试报告")
        print("=" * 40)
        print(f"总测试数: {total}")
        print(f"通过数: {self.test_results['passed']}")
        print(f"失败数: {self.test_results['failed']}")
        print(f"成功率: {success_rate:.1%}")
        
        if self.test_results['errors']:
            print("\n❌ 错误详情:")
            for error in self.test_results['errors']:
                print(f"  - {error}")
        
        if success_rate >= 0.8:
            status = "✅ 验证通过"
        elif success_rate >= 0.6:
            status = "⚠️ 部分通过"
        else:
            status = "❌ 验证失败"
        
        print(f"\n🏆 最终状态: {status}")
        return status, success_rate

# 运行验证测试
validator = IndicatorValidator(new_indicator)

print("🧪 开始验证测试...")
validator.test_basic_functionality(test_data)
validator.test_edge_cases(test_data) 
validator.test_performance(test_data)

# 生成测试报告
status, success_rate = validator.generate_report()

## 📊 可视化分析

In [ ]:
def plot_indicator_analysis(data, indicator_result, indicator_name="新指标"):
    """绘制指标分析图表"""
    
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    fig.suptitle(f'{indicator_name} - 综合分析', fontsize=16, fontweight='bold')
    
    # 1. 价格与指标对比
    ax1 = axes[0, 0]
    ax1_twin = ax1.twinx()
    
    ax1.plot(data.index, data['close'], label='收盘价', color='blue', linewidth=1.5)
    ax1_twin.plot(data.index, indicator_result, label=indicator_name, 
                  color='red', linewidth=1.5, alpha=0.8)
    
    ax1.set_title('价格 vs 指标')
    ax1.set_ylabel('价格', color='blue')
    ax1_twin.set_ylabel(indicator_name, color='red')
    ax1.legend(loc='upper left')
    ax1_twin.legend(loc='upper right')
    
    # 2. 指标分布直方图
    ax2 = axes[0, 1]
    indicator_clean = indicator_result.dropna()
    ax2.hist(indicator_clean, bins=30, alpha=0.7, color='orange', edgecolor='black')
    ax2.axvline(indicator_clean.mean(), color='red', linestyle='--', 
                label=f'均值: {indicator_clean.mean():.2f}')
    ax2.axvline(indicator_clean.median(), color='green', linestyle='--',
                label=f'中位数: {indicator_clean.median():.2f}')
    ax2.set_title('指标值分布')
    ax2.set_xlabel(indicator_name)
    ax2.set_ylabel('频次')
    ax2.legend()
    
    # 3. 相关性散点图
    ax3 = axes[1, 0]
    returns = data['close'].pct_change().dropna()
    indicator_aligned = indicator_result.reindex(returns.index).dropna()
    returns_aligned = returns.reindex(indicator_aligned.index)
    
    if len(indicator_aligned) > 0 and len(returns_aligned) > 0:
        correlation = np.corrcoef(returns_aligned, indicator_aligned)[0, 1]
        ax3.scatter(returns_aligned, indicator_aligned, alpha=0.6, s=20)
        ax3.set_title(f'收益率 vs 指标\n相关系数: {correlation:.3f}')
        ax3.set_xlabel('收益率')
        ax3.set_ylabel(indicator_name)
        
        # 添加趋势线
        z = np.polyfit(returns_aligned, indicator_aligned, 1)
        p = np.poly1d(z)
        ax3.plot(sorted(returns_aligned), p(sorted(returns_aligned)), "r--", alpha=0.8)
    
    # 4. 滚动统计
    ax4 = axes[1, 1]
    window = min(30, len(indicator_result) // 4)
    rolling_mean = indicator_result.rolling(window=window).mean()
    rolling_std = indicator_result.rolling(window=window).std()
    
    ax4.plot(data.index, rolling_mean, label=f'{window}日均值', color='blue')
    ax4.fill_between(data.index, rolling_mean - rolling_std, rolling_mean + rolling_std,
                     alpha=0.3, color='blue', label=f'±1标准差')
    ax4.plot(data.index, indicator_result, alpha=0.7, color='gray', linewidth=0.8)
    ax4.set_title(f'滚动统计 ({window}日)')
    ax4.set_ylabel(indicator_name)
    ax4.legend()
    
    # 5. 指标变化率
    ax5 = axes[2, 0]
    indicator_change = indicator_result.pct_change()
    ax5.plot(data.index, indicator_change, color='purple', alpha=0.8, linewidth=1)
    ax5.axhline(0, color='black', linestyle='-', alpha=0.3)
    ax5.set_title('指标变化率')
    ax5.set_ylabel('变化率')
    ax5.set_xlabel('日期')
    
    # 6. 统计摘要表
    ax6 = axes[2, 1]
    ax6.axis('off')
    
    # 计算统计指标
    stats = indicator_clean.describe()
    stats_text = f"""
统计摘要:
━━━━━━━━━━━━━━━━━━━━
样本数量: {stats['count']:.0f}
均值: {stats['mean']:.4f}
标准差: {stats['std']:.4f}
最小值: {stats['min']:.4f}
25%分位: {stats['25%']:.4f}
中位数: {stats['50%']:.4f}
75%分位: {stats['75%']:.4f}
最大值: {stats['max']:.4f}
━━━━━━━━━━━━━━━━━━━━
偏度: {indicator_clean.skew():.4f}
峰度: {indicator_clean.kurtosis():.4f}
空值比例: {indicator_result.isna().mean():.1%}
    """
    
    ax6.text(0.1, 0.9, stats_text, transform=ax6.transAxes, 
            fontfamily='monospace', fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return stats

# 绘制分析图表
if 'indicator_result' in locals():
    stats = plot_indicator_analysis(test_data, indicator_result, "新指标")
    print("📊 可视化分析完成")
else:
    print("⚠️ 请先成功计算指标结果")

## 🔗 集成测试

In [ ]:
def integrate_indicator_to_library():
    """将新指标集成到技术指标库"""
    
    print("🔗 准备集成新指标到技术指标库...")
    
    # 检查指标库当前状态
    ti = TechnicalIndicators()
    
    # 获取现有方法数量
    existing_methods = [method for method in dir(ti) if not method.startswith('_')]
    print(f"📊 当前指标库方法数: {len(existing_methods)}")
    
    # 准备添加新指标的代码
    new_indicator_code = f'''
    def {new_indicator.__name__}(self, data: pd.Series, param1: int = 14, param2: float = 2.0) -> pd.Series:
        """
        {new_indicator.__doc__ or '新开发的技术指标'}
        """
        # 指标实现代码
        result = data.rolling(window=param1).mean()
        return result
    '''
    
    print("\n📝 新指标代码预览:")
    print("=" * 50)
    print(new_indicator_code)
    print("=" * 50)
    
    # 集成建议
    integration_steps = [
        "1. 🧪 确认所有测试通过",
        "2. 📝 添加完整的文档字符串", 
        "3. 🔧 将代码添加到 TechnicalIndicators 类",
        "4. ✅ 更新 calculate_all_indicators 方法",
        "5. 📚 添加使用示例和单元测试",
        "6. 🚀 提交代码并更新版本号"
    ]
    
    print("\n🗺️ 集成步骤:")
    for step in integration_steps:
        print(f"  {step}")
    
    # 兼容性检查
    compatibility_checks = {
        '输入数据类型': 'pd.Series ✅',
        '返回数据类型': 'pd.Series ✅',
        '参数类型检查': '需要添加 ⚠️',
        '错误处理': '需要完善 ⚠️',
        '性能优化': '基本满足 ✅',
        '文档完整性': '需要补充 ⚠️'
    }
    
    print("\n🔍 兼容性检查:")
    for check, status in compatibility_checks.items():
        print(f"  {check}: {status}")
    
    return new_indicator_code, integration_steps

# 运行集成测试
if 'new_indicator' in locals():
    code, steps = integrate_indicator_to_library()
else:
    print("⚠️ 请先实现新指标函数")

## 🎯 开发总结

### ✅ 完成项目
- [ ] 指标功能实现
- [ ] 基础验证测试
- [ ] 性能基准测试  
- [ ] 可视化分析
- [ ] 集成准备

### 📊 指标质量评估
- **功能完整性**: ⭐⭐⭐⭐⭐ (5/5)
- **代码质量**: ⭐⭐⭐⭐⭐ (5/5) 
- **性能表现**: ⭐⭐⭐⭐⭐ (5/5)
- **文档完整性**: ⭐⭐⭐⭐⭐ (5/5)
- **测试覆盖**: ⭐⭐⭐⭐⭐ (5/5)

### 🚀 后续行动
1. **代码集成**: 将指标添加到主库
2. **文档更新**: 完善API文档和使用示例
3. **持续测试**: 建立持续集成测试
4. **社区反馈**: 收集用户反馈和改进建议

### 📝 开发笔记
记录开发过程中的重要发现、问题解决方案和优化思路...

---
**开发完成时间**: {completion_time}  
**状态**: ✅ 已完成 / 🔄 进行中 / ⏸️ 暂停 / ❌ 失败  
**下一个指标**: {next_indicator}